In [1]:
import torch

In [2]:
torch.randn(5,5).cuda()


-0.2146  0.5389 -2.2759 -0.1880 -0.3816
-1.3382 -0.1660 -0.2584  0.5645  0.3696
-0.1445 -2.2646  0.1299  1.2714  1.2196
-1.1038 -0.7197  1.0979 -0.6376 -0.6127
 0.0215 -0.7507  0.5784  0.9954 -2.5455
[torch.cuda.FloatTensor of size 5x5 (GPU 0)]

In [4]:
print("__Number CUDA Devices:", torch.cuda.device_count())

__Number CUDA Devices: 2


In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import re, string, sys
from operator import itemgetter
import matplotlib.pyplot as plt

In [3]:
sys.path.append('../tmp/bandit-nmt/')

In [4]:
from __future__ import division
import lib

In [5]:
dataset = torch.load('../tmp/bandit-nmt/data/en-zh/processed_all-train.pt')

In [8]:
valid_data = lib.Dataset(dataset["valid"], 64, [0], eval=True)

In [9]:
metrics = {}
metrics["nmt_loss"] = lib.Loss.weighted_xent_loss
metrics["critic_loss"] = lib.Loss.weighted_mse
metrics["sent_reward"] = lib.Reward.sentence_bleu
metrics["corp_reward"] = lib.Reward.corpus_bleu
dicts = dataset["dicts"]

In [20]:
checkpoint = torch.load('../tmp/bandit-nmt/scripts/log/model_15_pretrain.pt')
model = checkpoint["model"]
model = torch.nn.DataParallel(model, device_ids=[0, 1])
model.cuda(0)
optim = checkpoint["optim"]
start_epoch = checkpoint["epoch"] + 1
critic = checkpoint["critic"]
critic_optim = checkpoint["critic_optim"]

/home/ajanigyasi/tmp/anaconda3/lib/python3.6/site-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'lib.model.EncoderDecoder.Encoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [17]:
total_loss = 0
total_words = 0
total_sents = 0
total_sent_reward = 0

all_preds = []
all_targets = []

In [58]:
def _convert_and_report(data, pred_file, preds, metrics):
    preds = data.restore_pos(preds)
    with open(pred_file, "w") as f:
        for sent in preds:
            sent = lib.Reward.clean_up_sentence(sent, remove_unk=False, remove_eos=True)
            sent = [dicts["tgt"].getLabel(w) for w in sent]
            print(" ".join(sent), file=f)
    loss, sent_reward, corpus_reward = metrics
    print("")
    print("Loss: %.6f" % loss)
    print("Sentence reward: %.2f" % (sent_reward * 100))
    print("Corpus reward: %.2f" % (corpus_reward * 100))
    print("Predictions saved to %s" % pred_file)

In [18]:
for i in range(len(valid_data)):
    batch = valid_data[i]
    targets = batch[1]
    model.decoder.attn.applyMask(batch[0][0].data.eq(lib.Constants.PAD).t())
    outputs = model(batch, True)
    weights = targets.ne(lib.Constants.PAD).float()
    num_words = weights.data.sum()
    _, loss = model.predict(outputs, targets, weights, lib.Loss.weighted_xent_loss)
    
    preds = model.translate(batch, 50)
    preds = preds.t().tolist()
    targets = targets.data.t().tolist()
    rewards, _ = lib.Reward.sentence_bleu(preds, targets)
    
    all_preds.extend(preds)
    all_targets.extend(targets)

    total_loss += loss
    total_words += num_words
    total_sent_reward += sum(rewards)
    total_sents += batch[1].size(1)

loss = total_loss / total_words
sent_reward = total_sent_reward / total_sents
corpus_reward = lib.Reward.corpus_bleu(all_preds, all_targets)

print("")
print("Loss: %.6f" % loss)
print("Sentence reward: %.2f" % (sent_reward * 100))
print("Corpus reward: %.2f" % (corpus_reward * 100))


../tmp/bandit-nmt/lib/model/EncoderDecoder.py:24: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().
  outputs, hidden_t = self.rnn(emb, hidden)



Loss: 3.312458
Sentence reward: 24.36
Corpus reward: 14.46


In [22]:
preds = valid_data.restore_pos(all_preds)

for sent in preds:
    sent = lib.Reward.clean_up_sentence(sent, remove_unk=False, remove_eos=True)
    sent = [dicts["tgt"].getLabel(w) for w in sent]
print(sent)

['simple', '.']


In [11]:
segments_seen = {}
def evaluate(result, correct):
    result = result.split()
    print('Result:', result)
    if correct in segments_seen:
        correct = segments_seen[correct]
    else:
        segments_seen[correct] = \
        correct.split()
        correct = segments_seen[correct]
    print('Correct:', correct)
    score = lib.Bleu.score_sentence(correct, result, 1)
    print('Score:', score[-1])
    return score

In [23]:
score = evaluate('hello my name is crew', 'hello my name is crew')

Result: ['hello', 'my', 'name', 'is', 'crew']
Correct: ['hello', 'my', 'name', 'is', 'crew']
Score: 1.0


In [21]:
src, tgt, pred = [], [], []
sizes = []
count, ignored = 0, 0

In [27]:
src_valid = open('../tmp/bandit-nmt/data/en-zh/prep/valid.en-zh.zh')
tgt_valid = open('../tmp/bandit-nmt/data/en-zh/prep/valid.en-zh.en')
pred_valid = open('../tmp/bandit-nmt/scripts/log/model_15_pretrain.valid.pred')

while True:
    srcWords = src_valid.readline().split()
    tgtWords = tgt_valid.readline().split()
    predWords = pred_valid.readline().split()
    if not srcWords or not tgtWords:
        if srcWords and not tgtWords or not srcWords and tgtWords:
            print("WARNING: source and target do not have the same number of sentences")
        break

    # Only remove long sentences for training set.
    if len(srcWords) <= 50 and len(tgtWords) <= 50:
        src += [srcWords]
        tgt += [tgtWords]
        pred += [predWords]
        sizes += [len(srcWords)]
    else:
        ignored += 1

    count += 1
    if count % 100000 == 0:
        print("... %d sentences prepared" % count)

src_valid.close()
tgt_valid.close()
pred_valid.close()

score = lib.Bleu.score_sentence(pred[0], tgt[0],  1)
print('Source:', src[0])
print('Target:', tgt[0])
print('Prediction:', pred[0])
print('Score:', score[-1])

Source: ['它', '可以', '伸展', '到', '150', '英尺', '长', '。']
Target: ['it', 'gets', 'up', 'to', 'about', '150', 'feet', 'long', '.']
Prediction: ['you', 'see', 'a', 'team', 'of', 'teams', ',', 'students', 'in', 'drama', ',', 'school', 'students', ',', 'learning', 'english', ',', 'a', 'often', 'often', 'list', 'of', 'time', 'for', 'any', 'party', 'and', 'a', 'often', 'often', 'space', '.']
Score: 0.00862767574483065
